In [1]:
import numpy as np
from sklearn.datasets import load_boston

In [2]:
#type - Bunch
#Bunch - dictionary of numpy data
#boston.feature_names
#print(boston)
boston = load_boston()

In [3]:
X = boston.data
X.shape #number of samples, number of features

#====Step 1: Get your X and y in the right shape======

m = X.shape[0]  #number of samples
n = X.shape[1]  #number of features

In [4]:
y = boston.target

In [5]:
#number of rows in X is the same as number of rows in y
#because so we have yhat for all y
assert m == y.shape[0] 

In [6]:
#I want to standardize my data so that mean is 0, variance is 1
#average across each feature, NOT across each sample
#Why we need to standardize
#Because standardizing usually allows us to reach convergence faster
#Why -> because the values are within smaller range
#Thus, the gradients are also within limited range, and NOT go crazy

from sklearn.preprocessing import StandardScaler

#1. StandardScaler.fit(X)  #this scaler (or self) knows the mean and std so now
# it knows how to transform data
#2  X = StandardScaler.transform(X)  #not in place; will return something

#1. StandardScaler.fit_transform(X) -> 1 and 2 sequentially

#create an object of StandardScaler
#StandardScaler is a class
#scaler is called instance/object

#====Step 2: ALMOST always, feature scale your data using normalization or standardization
#If you assume your data is gaussian, use standardization, otherwise, you do the normalization

scaler = StandardScaler()

X = scaler.fit_transform(X)

In [7]:
#====Step 3: Train test split your data 

#what is the appropriate size for test data
#70/30 (small dataset); 80/20 (medium dataset); 90/10 (large dataset);
#why large dataset, can set test size to 10, because
#10% of large dataset is already enough for testing accuracy
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

assert len(X_train)  == len(y_train)
assert len(X_test) == len(y_test)

$$\theta = (X^TX)^{-1}X^TY$$

In [8]:
#Closed form
#How to get Closed Form
#Simple; Set the d(cost function) = 0
#And find the \theta that satisfy the equation
#When we can do such a thing in which we set the d(cost function) = 0
#--->When its strictly concave, or strictly convex
#----> They have only one local maximum (concave), minimum (convex)
#=====Step 4: Define your procedure to find theta ===

from numpy.linalg import inv

#run it, and return me the theta
#which one do first DOES NOT MATTER
#But don't flip y before X^T for example
def closed_form(X, y):
    return inv(X.T @ X) @ X.T @ y

In [9]:
#What is the shape of X they want
#(number of samples, number of features) --> correct shape
# for closed form formula
#How about the intercept
#w0 is OUR intercept
#what is the common shape of w -->(n+1, )
#What is the shape of 1 --->(m, 1)
#X = [1 2 3     @  [w0
#     1 4 6         w1
#     1 9 1         w2 
#     1 10 2 ] 

#np.ones((shape))
intercept = np.ones((X_train.shape[0], 1))

#concatenate the intercept based on axis=1
X_train = np.concatenate((intercept, X_train), axis=1)

#np.ones((shape))
intercept = np.ones((X_test.shape[0], 1))

#concatenate the intercept based on axis=1
X_test = np.concatenate((intercept, X_test), axis=1)

In [10]:
#=====Step 5: Find the theta/weights/coefficients/beta=====
w = closed_form(X_train, y_train)
w  #<------somehow is our model

array([22.7281486 , -1.03784848,  1.10085393,  0.05958242,  0.98754043,
       -2.18106046,  3.13039422,  0.12442295, -3.36474438,  2.76242277,
       -1.99224799, -1.89946081,  0.88911465, -3.71947443])

In [11]:
#====Step 6: Compute the accuracy/loss

#6.1 predict --> \theta^T x
yhat = X_test @ w #==> X (m, n+1)  @ (n+1, ) w ==> (m, ) y

#if I want to compare yhat and y, I need to make sure they are the same shape
assert y_test.shape == yhat.shape

In [12]:
#6.2. ---> get the errors
errors = ((y_test - yhat)**2).sum()

In [13]:
print(errors)

3456.2815160721893


In [14]:
X_train.shape

(354, 14)

In [15]:
y_train.shape

(354,)

In [16]:
#Step 1: Prepare your data
#X_train, X_test have intercepts that are being concatenated to the data
#[1, features
# 1, features....]

#making sure our X_train has same sample size as y_train
assert X_train.shape[0] == y_train.shape[0]

#initialize our w
#We don't have to do X.shape[1] + 1 because our X_train already has the
#intercept
#w = theta/beta/coefficients
w = np.zeros(X_train.shape[1])

#define the learning rate
#later on, you gonna know that it should be better to make it slowly decreasing
#once we perform a lot of iterations, we want the update to slow down, so it converges better
alpha = 0.0001

#define our max_iter
#typical to call it epochs <---ml people likes to call it
max_iter = 1000

loss_old = 10000

tol = 0.0001

iter_stop = 0

def get_yhat(X, w):
    return X @ w

def get_mse(yhat, y):
    return ((yhat - y)**2 / yhat.shape[0]).sum()

def delta_loss(new, old, tol):
    return np.abs(loss_new - loss_old) < tol

def gradient(X, error):
    return X.T @ error

#define your for loop
for i in range(max_iter):
    
    #1. yhat = X @ w
    #prediction
    #yhat (m, ) = (m, n) @ (n, )
    yhat = get_yhat(X_train, w)

    #2. error = yhat - y_train
    #error for use to calculate gradients
    #error (m, ) = (m, ) - (m, )
    error = yhat - y_train

    #2.1 early stopping
    #so we don't go through all max_iter iterations
    # )yi_hat - yi )^2 / m  <--- mse
    #loss_new (scalar) = ((m, ) - (m, ) **2 / m).sum()
    loss_new = get_mse(yhat, y_train)
    if delta_loss(loss_new, loss_old, tol):  #np.allclose
        iter_stop = i
        break
    loss_old = loss_new

    #3. grad = X.T @ error
    #grad (n, ) = (n, m) @ (m, )
    #grad for each feature j
    grad = gradient(X_train, error)

    #4. w = w - alpha * grad
    #update w
    #w (n, ) = (n, ) - scalar * (n, )
    w = w - alpha * grad


#we got our lovely w
#now it's time to check our accuracy
#1. Make prediction
yhat = get_yhat(X_test, w)

#2. Calculate mean squared errors
mse = get_mse(yhat, y_test)

#print the mse
print("MSE: ", mse)
print("Stop at iteration: ", iter_stop)

MSE:  22.786395077682954
Stop at iteration:  741


In [17]:
#Stochastic version

def h_theta_x(X, w):
    '''
    Input:
        X shape (m, n)
        w shape (n, )
    Returns: 
        (m, )
    '''
    return X @ w

def get_mse(yhat, y):
    return ((yhat - y)**2 / yhat.shape[0]).sum()

def get_singe_mse(yhat, y):
    #yhat (1, ) - (1, ) ** 2
    return (yhat - y)**2

def delta_loss(new, old, tol):
    return np.abs(loss_new - loss_old) < tol

def gradient(X, error):
    return X.T @ error

#initialize our w
#We don't have to do X.shape[1] + 1 because our X_train already has the
#intercept
#w = theta/beta/coefficients
theta = np.zeros(X_train.shape[1])

#define the learning rate
#later on, you gonna know that it should be better to make it slowly decreasing
#once we perform a lot of iterations, we want the update to slow down, so it converges better
alpha = 0.01
loss_old = 10000
tol = 0.0001
iter_stop = 0
max_epochs = 10000


#define your for loop
for epoch in range(max_epochs):  #max_iter is the same as epochs

    #we have indices for all samples
    i = np.random.randint(X_train.shape[0])
    
    #1. yhat = X_i @ w
    #X_i (1, n)
    X_i = X_train[i, :].reshape(1, -1)

    #prediction
    #yhat (1, ) = (1, n) @ (n, )
    yhat = h_theta_x(X_i, theta)

    #2. error = yhat - y_i
    #y_i (1, )
    y_i = y_train[i]
    #error for use to calculate gradients
    #error (1, ) = (1, ) - (1, )
    error = yhat - y_i

    #2.1 early stopping
    #so we don't go through all max_iter iterations
    # (yi_hat - yi )^2 / m  <--- mse
    #loss_new (scalar) = ((m, ) - (m, ) **2 / m).sum()
    loss_new = get_singe_mse(yhat, y_i)
    if delta_loss(loss_new, loss_old, tol):  #np.allclose
        iter_stop = epoch
        break
    loss_old = loss_new

    #3. grad = X.T @ error
    #grad (n, ) = (n, 1) @ (1, )
    #grad for each feature j
    grad = gradient(X_i, error)

    #4. w = w - alpha * grad
    #update w
    #w (n, ) = (n, ) - scalar * (n, )
    theta = theta - alpha * grad


#we got our lovely w
#now it's time to check our accuracy
#1. Make prediction
yhat = h_theta_x(X_test, theta)

#2. Calculate mean squared errors
mse = get_mse(yhat, y_test)

#print the mse
print("MSE: ", mse)
print("Stop at iteration: ", iter_stop)

MSE:  26.42954342037887
Stop at iteration:  0
